In [1]:
from google.colab import drive 
drive.mount('/content/drive')
import pandas as pd 
import numpy as np
import math

Mounted at /content/drive


In [2]:
Data_lista= pd.read_excel('/content/drive/MyDrive/TRABAJO/Autocoevaluación/Autocoevaluación/Documentos/Lista.xlsx')
Data_Respuestas= pd.read_excel('/content/drive/MyDrive/TRABAJO/Autocoevaluación/Autocoevaluación/Documentos/Respuestas.xlsx')
Data_Respuestas = Data_Respuestas.rename(columns={'¿A qué equipo pertenece?':'Equipo'})
Data_lista.head()
Subdata_Cedulas = Data_lista['Documento'].to_list()
Subdata_Nombre = Data_lista['Nombre'].to_list()
Subdata_Equipos_lista = Data_lista['Equipo'].to_list()
# Diccionario que contiene como clave la cedula y como valor el nombre
Dict_Cedulas = {}
for i in Subdata_Cedulas :
  Dict_Cedulas[i]= Subdata_Nombre[Subdata_Cedulas.index(i)]



#Cantidad de personas que hay en cada equipo 
Dict_Personas = {}
for i in Subdata_Equipos_lista :
  if i in list(Dict_Personas.keys()) :
    Dict_Personas[i] += 1
  else:
    Dict_Personas[i] = 1


#Diccionario de diccionarios 
Equipos= {}
Cedulas = Data_Respuestas['Cédula'].to_list()
Nombres_Equipos = []
Subdata_Equipos = Data_Respuestas['Equipo']
for i in Subdata_Equipos :
  claves = list(Equipos.keys())
  if i not in claves:
    Equipos[i] = {}
    Nombres_Equipos.append(i)
#aqui vamos


for nombre in Nombres_Equipos :
  #Equipo es un diccionario
  Equipo = Equipos[nombre]
  #Data que solo tiene los integrantes de un equipo 
  Subdata_Equipo_Actual = Data_Respuestas[Data_Respuestas.Equipo == nombre]
  Lista_cedulas = Subdata_Equipo_Actual['Cédula'].to_list()

  Subdata_Equipo_Actual_Totales = Data_lista[Data_lista.Equipo == nombre]
  Lista_cedulas_Totales = Subdata_Equipo_Actual_Totales['Documento'].to_list()
  

  for estudiante in Lista_cedulas_Totales:
      
    #Dict_Actual = Equipo[Dict_Cedulas[estudiante]]
    Dict_Actual = {}
    
    for cedula in Lista_cedulas_Totales:
      nombre_buscar =str(Dict_Cedulas[cedula]) 
      if estudiante in Lista_cedulas:
        index = Cedulas.index(estudiante)
        celda = Data_Respuestas.at[index,nombre_buscar]
      else:
        celda = 0.0
      Dict_Actual[nombre_buscar]=  celda
      
    Equipo[Dict_Cedulas[estudiante]] = Dict_Actual

   
#Cuando pueda modificar lo de que los valores sean muy diferentes de 100 en total 
Dict_Total = {}
Dict_Total_2 = {}
Dict_Promedio ={}
Dict_Desviacion = {}
Dict_Maximo = {}
Dict_Opinion = {}
Dict_Ajuste_Ego = {}
Dict_Proporcion = {}
Dict_Asertividad = {}
Dict_Ajuste_Part= {}



def Repartir_Valores(repartir,nombre,Dict_Equipo_Actual):
  Dict_Modificar = Dict_Equipo_Actual[nombre]
  
  for i in Dict_Modificar.keys():
    Dict_Modificar[i] += repartir

def Repartir_Valores_Ego(proporcion,nombre,Dict_Equipo_Actual):
  
  Dict_Modificar = Dict_Equipo_Actual[nombre]
  
  for i in Dict_Modificar.keys():
    if i != nombre :
      
      Dict_Modificar[i] += proporcion
    
def Cambio_Max_Ego(Max,nombre,Dict_Equipo_Actual):
  Dict_Modificar = Dict_Equipo_Actual[nombre]
  Dict_Modificar[nombre] = Max

  


  
for equipo in Equipos.keys() :
  Dict_Equipo_Actual = Equipos[equipo]
  #print(type(Dict_Equipo_Actual.keys()))
  Dict_provisional = {}
  for nombre in Dict_Equipo_Actual.keys():
    valores = Dict_Equipo_Actual[nombre].values()
    total = 0
   
    
    for j in valores :
      total += j
    Dict_provisional[nombre] = total
    diferencia = 100- total

    
    if diferencia > 0 :
      #print(Dict_Equipo_Actual[nombre])
      
      repartir = diferencia/Dict_Personas[equipo]
      Repartir_Valores(repartir,nombre,Dict_Equipo_Actual)
      Dict_provisional[nombre] = 100
      #print(Dict_Equipo_Actual[nombre])
      #print(nombre)
      #print(diferencia)
      #print(repartir)
      

  
  Dict_Total[equipo] = Dict_provisional



for equipo in Equipos.keys() :
  #Diccionarios
  Dict_Equipo_Actual = Equipos[equipo]
  Dict_provisional = {}
  Dict_provisional_prom = {}
  Dict_provisional_desv = {}
  Dict_provisional_max ={}
  Dict_provisional_opi = {}
  Dict_provisional_Ajuste_Ego = {}
  Dict_provisional_prop = {}
  Dict_provisional_Aser = {}
  Dict_provisional_Ajuste_Part = {}
  #Lista
  Lista_provisional =[]
  for nombre in Dict_Equipo_Actual.keys():
    total = 0
    for i in Dict_Equipo_Actual.keys():  
      Dict = Dict_Equipo_Actual[i]  
      valor = Dict[nombre]

      Lista_provisional.append(valor)
      total += valor
      if nombre == i :
        Dict_provisional_opi[nombre] = valor

        
    
    prom=np.mean(Lista_provisional)
    desv=np.std(Lista_provisional)
    Lista_provisional.clear()
    Max = prom + desv
    Opinion_Actual = Dict_provisional_opi[nombre]
    Dict_provisional[nombre] = total
  
    Dict_provisional_prom[nombre] = prom
    Dict_provisional_desv[nombre] = desv
    Dict_provisional_max[nombre] = Max
    Ajuste_Ego = Max - Opinion_Actual
    proporcion = -(Ajuste_Ego/(Dict_Personas[equipo]-1))
    if Opinion_Actual > Max :     
      Dict_provisional_Ajuste_Ego[nombre] = Ajuste_Ego      
      Dict_provisional_prop[nombre] = proporcion
    else:
      Dict_provisional_Ajuste_Ego[nombre]=0
      Dict_provisional_prop[nombre] = 0

    Cambio_Max_Ego(Max,nombre,Dict_Equipo_Actual)
    
    Repartir_Valores_Ego(proporcion,nombre,Dict_Equipo_Actual)

    Asertividad= math.pow((100-desv)/100, 4) 
    Ajuste_Asertividad = (1-Asertividad)/(Dict_Personas[equipo]-1)

    Dict_provisional_Aser[nombre] = Asertividad
    Dict_provisional_Ajuste_Part[nombre] = Ajuste_Asertividad

  Dict_Total_2[equipo] = Dict_provisional
  Dict_Promedio[equipo] = Dict_provisional_prom
  Dict_Desviacion[equipo] = Dict_provisional_desv
  Dict_Maximo[equipo] = Dict_provisional_max
  Dict_Opinion[equipo] = Dict_provisional_opi
  Dict_Ajuste_Ego[equipo] = Dict_provisional_Ajuste_Ego
  Dict_Proporcion[equipo] = Dict_provisional_prop
  Dict_Ajuste_Part[equipo] = Dict_provisional_Ajuste_Part
  
  for nombre in Dict_provisional_Aser :
    for ajuste in Dict_provisional_Ajuste_Part :
      if nombre != ajuste:
        Dict_provisional_Aser[nombre] += Dict_provisional_Ajuste_Part[ajuste]
        
  Dict_Asertividad[equipo] = Dict_provisional_Aser

Dict_Resultado_Final = {}
for equipo in Equipos.keys() :
  
  Dict_Equipo_Actual = Equipos[equipo]
  Dict_Asertividad_Actual = Dict_Asertividad[equipo]
  Dict_provisional = {}
  for nombre in Dict_Equipo_Actual.keys():
    total=0
    
    for nombre2 in Dict_Equipo_Actual.keys():
      
      Dict = Dict_Equipo_Actual[nombre2]  
      valor_Aser = Dict_Asertividad_Actual[nombre2]
      valor = Dict[nombre]
      
      total += valor * valor_Aser
    Dict_provisional[nombre] = round((total/100 ),2)

  Dict_Resultado_Final[equipo] = Dict_provisional


for equipo in Dict_Resultado_Final.keys() :
  Dict_Equipo_Actual = Dict_Resultado_Final[equipo]
  print(equipo)
  for nombre in Dict_Equipo_Actual.keys():
    print(nombre + ": " + str(Dict_Equipo_Actual[nombre]))
  print("\n")



Equipo 4
Asaf Joel Aguilar Lema: 0.94
Camila Andrea Amín Puello: 1.09
Natalia Carrejo Muñoz: 1.08
Nicolas Sebastian Guatibonza Pardo: 0.96
Maria Jose Ortiz Pastrana: 0.93
Manuela Taborda Monsalve: 1.04
Daniel Alexander Villada Bedoya: 0.96


Equipo 5
Alexander Fonseca Cipriani: 1.14
Amalyn Katherine Muchavisoy Chindoy: 0.96
Juan David Ordoñez Noriega: 0.88
Santiago Restrepo Parra: 0.97
Pedro Daniel Torres Arias: 1.0
Natalia Victoria Giraldo: 1.05


Equipo 7
Juan Camilo Aguilar Maya: 1.39
Jaime Antonio Arango Betancur: 0.42
Lina Alejandra Arcos Hurtado: 0.75
Santiago Cuervo Ortega: 1.36
Sebastian Alexander Dorado Pazos: 1.44
Nelson David Gutiérrez Tangarife: 1.02
Carlos Alberto Rincon Florez: 0.62


Equipo 3
Marcos Arango Tamayo: 0.97
Carlos Daniel Bolivar Zapata: 0.96
Santiago Jiménez Gómez: 1.04
Shorly López Pérez: 0.98
Alejandro Morales Galeano: 1.03
María Camila Ramirez Trujillo: 0.93
Juan David Revelo Bucheli: 1.08


Equipo 1
Annette Gallego Sánchez: 0.98
Jhon Alfonso Latorre Prada